# CSIE 5452, Fall 2019 — Homework 2

In [1]:
import sys
import math
import copy
import time
import random
import numpy as np
from pandas.core.frame import DataFrame

In [2]:
def preprocess_data(file_dir):
    
    with open(file_dir, 'r') as f:
        dat = [line.rstrip('\n') for line in f]
    msg_num = int(dat[0])
    one_bit_trans = float(dat[1])
              
    temp = []
    for i in range (len(dat) - 2):
        temp.append(dat[i+2])
    
    data = []
    i = 0
    for d in temp:
        data.append(d.split(' '))
        if '' in data[i]:
            data[i].remove('')
        for j, item  in  enumerate(data[i]):
            try:
                data[i][j] = int(data[i][j])
            except ValueError:
                data[i][j] = float(data[i][j])
        i+=1
    
    return msg_num, one_bit_trans, data

In [3]:
def RHS(i, Qi, Bi, tao, data):
    
    temp = 0
    for n in range (i):
        temp += math.ceil(((Qi + tao)/data[n][2]))*data[n][1]
    rhs = Bi + temp
    
    return rhs

In [4]:
def calQi(i, q, Bi, Ci, Ti, tao, data):
    
    rhs = RHS(i, q, Bi, tao, data)
    if rhs + Ci >Ti:
        return 'nan'
    elif rhs == q:
        return q
    else:
        return calQi(i, rhs, Bi, Ci, Ti, tao, data)

In [5]:
def calBi(i, data):
    
    blocking_time = data[i][1]
    for j in range(len(data) - (i+1)):
        if data[i+j][1] > blocking_time:
            blocking_time = data[i+j][1]
    
    return blocking_time

In [6]:
def calRi(i, Bi, tao, data):
    
    if i == 0:
        Q = Bi
        R = Q + data[i][1]
        return R
    else:
        Q = calQi(i, Bi, Bi, data[i][1], data[i][2], tao, data)
        if Q == 'nan':
            return 'nan'
        else:
            R = Q + data[i][1]        
            return R

In [7]:
def Ri_List(data_list):
    Ri_list = []
    for i, item in enumerate(data_list):
        Bi = calBi(i, data_list)
        Ri = calRi(i, Bi, TAO, data_list)
        if Ri == 'nan':
            #print('The system is not schedulable')
            return 'nan'
            break
        
        else:
            Ri_list.append(Ri)
            #print ('Message μ'+ str(i) + ' worst-case response time: ' + str(Ri))
    return Ri_list  

In [8]:
FILE_DIR = 'Input.dat'
MSG_NUM, TAO, DATA = preprocess_data(FILE_DIR)

In [9]:
Ri_LIST = Ri_List(DATA)
Ri_LIST

[1.44,
 2.04,
 2.56,
 3.16,
 3.68,
 4.28,
 5.2,
 8.4,
 9.0,
 9.68,
 10.2,
 19.360000000000003,
 19.8,
 20.32,
 29.400000000000002,
 29.76,
 30.28]

In [10]:
def cost_func(list):
    SUM = sum(list)
    return SUM

In [11]:
def reverse(list, r1, r2):
    temp = list[min(r1, r2):max(r1, r2)]
    temp.reverse()
    list[min(r1, r2):max(r1, r2)] = temp
    return list

In [12]:
def new(L):
    l = list(range(0,len(L)))
    #print(l)
    random.shuffle(l)
    #print(l)
    r1 = l[0]
    r2 = l[1]
    #print(r1, r2)
    new_data = reverse(L, r1, r2)
    return new_data

In [13]:
init_T = 1000  # 初始溫度
min_T = 1  # 溫度下限
iterL = 10  # 每個T值的迭代次數
delta = 0.95  # 溫度衰減係數
k = 1

init_S = cost_func(Ri_LIST)
print (init_S)
curr_T = init_T
curr_Data = copy.deepcopy(DATA)

208.56


In [14]:
ALL_LIST = []
ALL_LIST.append(curr_Data)
dup_Data = copy.deepcopy(curr_Data)
tStart = time.time()
time.sleep(2)
while curr_T > min_T:
    for i in range (iterL):
        #print(dup_Data)
        new_data_list = new(dup_Data)
        #print(new_data_list)
        new_Ri_LIST = Ri_List(new_data_list)
        if new_Ri_LIST == 'nan':
            dup_Data = copy.deepcopy(ALL_LIST[len(ALL_LIST)-1])
            #print('Failed')
            break
        else:
            new_S = cost_func(new_Ri_LIST)
            res = new_S - init_S
            if res < 0:
                init_S = new_S
            else:
                p = np.exp(-res / (k * curr_T))
                if np.random.rand() < p:
                    init_S = new_S
            ALL_LIST.append(new_data_list)
            dup_data = copy.deepcopy(new_data_list) 
            print('Successed')
        #print(i)
        if len(ALL_LIST)%10 == 0:
            curr_T = curr_T * delta
            break
        print(curr_T)
tEnd = time.time()
print (init_S)
print ('It cost %f sec' %(tEnd - tStart))

Successed
1000
Successed
1000
Successed
1000
Successed
1000
Successed
1000
Successed
1000
Successed
1000
Successed
1000
Successed
Successed
950.0
Successed
950.0
Successed
950.0
Successed
950.0
Successed
950.0
Successed
950.0
Successed
950.0
Successed
950.0
Successed
950.0
Successed
Successed
902.5
Successed
902.5
Successed
902.5
Successed
902.5
Successed
902.5
Successed
902.5
Successed
902.5
Successed
902.5
Successed
902.5
Successed
Successed
857.375
Successed
857.375
Successed
857.375
Successed
857.375
Successed
857.375
Successed
857.375
Successed
857.375
Successed
857.375
Successed
857.375
Successed
Successed
814.5062499999999
Successed
814.5062499999999
Successed
814.5062499999999
Successed
814.5062499999999
Successed
814.5062499999999
Successed
814.5062499999999
Successed
814.5062499999999
Successed
814.5062499999999
Successed
814.5062499999999
Successed
Successed
773.7809374999998
Successed
773.7809374999998
Successed
773.7809374999998
Successed
773.7809374999998
Successed
773.78

Successed
184.02591023557582
Successed
184.02591023557582
Successed
184.02591023557582
Successed
Successed
174.82461472379703
Successed
174.82461472379703
Successed
174.82461472379703
Successed
174.82461472379703
Successed
174.82461472379703
Successed
174.82461472379703
Successed
174.82461472379703
Successed
174.82461472379703
Successed
174.82461472379703
Successed
Successed
166.08338398760716
Successed
166.08338398760716
Successed
166.08338398760716
Successed
166.08338398760716
Successed
166.08338398760716
Successed
166.08338398760716
Successed
166.08338398760716
Successed
166.08338398760716
Successed
166.08338398760716
Successed
Successed
157.7792147882268
Successed
157.7792147882268
Successed
157.7792147882268
Successed
157.7792147882268
Successed
157.7792147882268
Successed
157.7792147882268
Successed
157.7792147882268
Successed
157.7792147882268
Successed
157.7792147882268
Successed
Successed
149.89025404881545
Successed
149.89025404881545
Successed
149.89025404881545
Successed
14

Successed
33.86553563803219
Successed
33.86553563803219
Successed
33.86553563803219
Successed
33.86553563803219
Successed
33.86553563803219
Successed
33.86553563803219
Successed
33.86553563803219
Successed
33.86553563803219
Successed
Successed
32.17225885613058
Successed
32.17225885613058
Successed
32.17225885613058
Successed
32.17225885613058
Successed
32.17225885613058
Successed
32.17225885613058
Successed
32.17225885613058
Successed
32.17225885613058
Successed
32.17225885613058
Successed
Successed
30.563645913324052
Successed
30.563645913324052
Successed
30.563645913324052
Successed
30.563645913324052
Successed
30.563645913324052
Successed
30.563645913324052
Successed
30.563645913324052
Successed
30.563645913324052
Successed
30.563645913324052
Successed
Successed
29.035463617657847
Successed
29.035463617657847
Successed
29.035463617657847
Successed
29.035463617657847
Successed
29.035463617657847
Successed
29.035463617657847
Successed
29.035463617657847
Successed
29.035463617657847
S

Successed
6.232136021404204
Successed
6.232136021404204
Successed
6.232136021404204
Successed
6.232136021404204
Successed
6.232136021404204
Successed
6.232136021404204
Successed
6.232136021404204
Successed
6.232136021404204
Successed
6.232136021404204
Successed
Successed
5.920529220333994
Successed
5.920529220333994
Successed
5.920529220333994
Successed
5.920529220333994
Successed
5.920529220333994
Successed
5.920529220333994
Successed
5.920529220333994
Successed
5.920529220333994
Successed
5.920529220333994
Successed
Successed
5.624502759317294
Successed
5.624502759317294
Successed
5.624502759317294
Successed
5.624502759317294
Successed
5.624502759317294
Successed
5.624502759317294
Successed
5.624502759317294
Successed
5.624502759317294
Successed
5.624502759317294
Successed
Successed
5.3432776213514295
Successed
5.3432776213514295
Successed
5.3432776213514295
Successed
5.3432776213514295
Successed
5.3432776213514295
Successed
5.3432776213514295
Successed
5.3432776213514295
Successed
5

Successed
Successed
1.1468745040508286
Successed
1.1468745040508286
Successed
1.1468745040508286
Successed
1.1468745040508286
Successed
1.1468745040508286
Successed
1.1468745040508286
Successed
1.1468745040508286
Successed
1.1468745040508286
Successed
1.1468745040508286
Successed
Successed
1.089530778848287
Successed
1.089530778848287
Successed
1.089530778848287
Successed
1.089530778848287
Successed
1.089530778848287
Successed
1.089530778848287
Successed
1.089530778848287
Successed
1.089530778848287
Successed
1.089530778848287
Successed
Successed
1.0350542399058726
Successed
1.0350542399058726
Successed
1.0350542399058726
Successed
1.0350542399058726
Successed
1.0350542399058726
Successed
1.0350542399058726
Successed
1.0350542399058726
Successed
1.0350542399058726
Successed
1.0350542399058726
Successed
205.72
It cost 13.804656 sec


In [15]:
dataframe = DataFrame(new_data_list)
dataframe.rename(columns={0:'Orginal Priority',1:'Trans. Time',2:'Period'},inplace=True)
dataframe

,Orginal Priority,Trans. Time,Period
0,3,0.60,5
1,5,0.60,5
2,2,0.52,5
3,4,0.52,5
4,7,0.52,10
5,1,0.60,5
6,8,0.60,10
7,6,0.92,10
8,0,0.52,50
9,9,0.68,10


In [16]:
new_Ri_LIST

[1.52,
 2.12,
 2.64,
 3.16,
 3.68,
 4.28,
 4.88,
 5.800000000000001,
 9.0,
 9.68,
 10.2,
 19.12,
 19.8,
 20.32,
 29.240000000000002,
 30.000000000000004,
 30.28]

In [17]:
dataframe = DataFrame(DATA)
dataframe.rename(columns={0:'Orginal Priority',1:'Trans. Time',2:'Period'},inplace=True)
dataframe

,Orginal Priority,Trans. Time,Period
0,0,0.52,50
1,1,0.60,5
2,2,0.52,5
3,3,0.60,5
4,4,0.52,5
5,5,0.60,5
6,6,0.92,10
7,7,0.52,10
8,8,0.60,10
9,9,0.68,10
